In [5]:
import time
from bokeh.plotting import figure, show, Session, cursession, output_server
from data import datagenerator, datahandler, datastream

In [6]:
session = Session(load_from_config=False, root_url = "http://127.0.0.1:5006/")
output_server("barchart", url = "http://127.0.0.1:5006/")

Using saved session configuration for http://127.0.0.1:5006/
To override, pass 'load_from_config=False' to Session


In [7]:
#Inizializzazione variabili query
#GOOGL, ENEL.MI, UCG.MI, ISP.MI
x_quote_id = "UCG.MI"
y_quote_id = "ISP.MI"
start_y = 2014
start_m = 1
start_d = 1
stop_y = 2015
stop_m = 6
stop_d = 30
step = "d"

In [8]:
x_file = datastream.queryinput_to_csv_yf(x_quote_id, start_y, start_m, start_d, stop_y, stop_m, stop_d, step)
y_file = datastream.queryinput_to_csv_yf(y_quote_id, start_y, start_m, start_d, stop_y, stop_m, stop_d, step)

In [9]:
x_file.readline()
y_file.readline()

b'Date,Open,High,Low,Close,Volume,Adj Close\n'

In [10]:
x_linelist = x_file.readlines()
y_linelist = y_file.readlines()

In [11]:
#print(x_file.geturl());print( y_file.geturl())
#print(len(x_linelist), len(y_linelist))

In [12]:
x_file.close()
y_file.close()

In [13]:
x_nextlastline_gen = datahandler.yield_from_last(x_linelist)
y_nextlastline_gen = datahandler.yield_from_last(y_linelist)

In [14]:
x_data0 = datastream.split_csvline_yf(next(x_nextlastline_gen))
y_data0 = datastream.split_csvline_yf(next(y_nextlastline_gen))

##se le date relative alle ultime linee delle due liste non combaciano, allora i successivi while entrano in azione
##viceversa, i cicli while non alterano in alcun modo il contenuto di x_data0 e y_data0
##print(x_linelist[-1])
##print(y_linelist[-1])
##qui controllo che split_csvline_yf() prenda il dato corretto dalla linea esso passata
##print(x_data0[6], y_data0[6]) 
##print()

#i due cicli while scartano linee da x_linelist o y_linelist finche le due linee correnti non hanno la stessa data
#i dati sono scartati dalla lista che presenta data precendente
##i=0; j=0 #contano quanti dati sono stati scartati da x_linelist (i) o y_linelist (j) prima di avere sinc temporale
while x_data0[0]<y_data0[0]: x_data0 = datastream.split_csvline_yf(next(x_nextlastline_gen))##;i+=1
while y_data0[0]<x_data0[0]: y_data0 = datastream.split_csvline_yf(next(y_nextlastline_gen))##;j+=1

##indicizzazione effettuata in modo da trovare la linea con data uguale alla linea dell'altra lista. Se al passo
##precedente le date delle due linee non combaciavano, qui gli indici di accesso alle due liste saranno diversi
##print(x_linelist[-1-i])
##print(y_linelist[-1-j])
##print(x_data0[6], y_data0[6])
##print()

x_data = (datastream.split_csvline_yf(next(x_nextlastline_gen)))
y_data = (datastream.split_csvline_yf(next(y_nextlastline_gen)))

##come sopra, ma per x_data e y_data
##print(x_linelist[-2-i])
##print(y_linelist[-2-j])
##print(x_data[6], y_data[6])
##print()

while x_data[0]<y_data[0]: x_data = datastream.split_csvline_yf(next(x_nextlastline_gen));##i+=1
while y_data[0]<x_data[0]: y_data = datastream.split_csvline_yf(next(y_nextlastline_gen));##j+=1
    
##print(x_linelist[-2-i])
##print(y_linelist[-2-j])
##print(x_data[6], y_data[6])

In [15]:
##calcola la differenza tra l'adjusted closure del giorno attuale e quella del giorno precedente per titolo x e y
##print(x_data[6], x_data0[6])
##print(y_data[6], y_data0[6])
x_deltaclose = [x_data[6] - x_data0[6]]
y_deltaclose = [y_data[6] - y_data0[6]]
##print(x_deltaclose, y_deltaclose); #verifica che le differenze tornino

In [16]:
p = figure(
    plot_width = 800,
    plot_height = 800,
)
p.xaxis.axis_label = x_quote_id
p.yaxis.axis_label = y_quote_id

In [17]:
p.circle(
    x = x_deltaclose, #come primo dato, assegna ad x e y del cerchio i delta adj close dei due titoli
    y = y_deltaclose,
    size = 10,
    color = "navy",
    alpha = 0.5,
    name = "scatter"
)

In [18]:
show(p)

ds_scatter = p.select({"name":"scatter"})[0].data_source

##assegna al giorno precedente i dati finora assegnati al giorno attuale, shiftando di 1 in avanti l'asse dei tempi
x_data0 = x_data
y_data0 = y_data
##controlla che i precedenti x_data[6], y_data[6] siano gli attuali x_data0[6], y_data0[6]
##print("\n%r \n%r" %(x_data0, y_data0)) 

In [19]:
##iter_n = 0 ##contatore del numero di iterazioni del ciclo while utilizzato in fase di debugging

while True:
    
    try:
        x_data = datastream.split_csvline_yf(next(x_nextlastline_gen))
        y_data = datastream.split_csvline_yf(next(y_nextlastline_gen))

        while x_data[0] < y_data[0]: x_data = datastream.split_csvline_yf(next(x_nextlastline_gen))##;i+=1
        while y_data[0] < x_data[0]: y_data = datastream.split_csvline_yf(next(y_nextlastline_gen))##;j+=1
        
        ##controllo che tutti i dati siano relativi alla stessa data
        ##print(x_data[0], y_data[0], x_data0[0] , y_data0[0]) 
        
        ##controllo che vi sia coerenza tra i delta adj_close ottenuti direttamente dalle
        ##dai dati riportati nel csv e i delta adj_close plottati
        ##NB: per come aggiorno i dati pescati dalle liste, è normale che in questo controllo
        ##le due differenze non siano uguali quando si è verificato uno shift delle date
        ##print("\n\n%r \n%r" %(x_linelist[-iter_n-1-i], x_linelist[-iter_n-2-i]))
        ##print("%r %r" %(x_deltaclose[-1],
        ##    -datastream.split_csvline_yf(x_linelist[-iter_n-1-i])[6] 
        ##    + datadstream.split_csvline_yf(x_linelist[-iter_n-2-i])[6]))
        ##print("\n%r \n%r" %(y_linelist[-iter_n-1-j], y_linelist[-iter_n-2-j]))
        ##print("%r %r" %(y_deltaclose[-1],
        ##    -datastream.split_csvline_yf(y_linelist[-iter_n-1-j])[6] 
        ##    + datastream.split_csvline_yf(y_linelist[-iter_n-2-j])[6]))
        ##iter_n += 1
        
        x_deltaclose.append(x_data[6] - x_data0[6])
        y_deltaclose.append(y_data[6] - y_data0[6])

        x_data0 = x_data
        y_data0 = y_data
        
    except(StopIteration):
        ##print("Dati terminati")
        ##print("Numero punti plottati: %r" %len(x_deltaclose))
        break
        
    ds_scatter.data["x"] = x_deltaclose
    ds_scatter.data["y"] = y_deltaclose
    
    cursession().store_objects(ds_scatter)
    time.sleep(0.05)

KeyboardInterrupt: 

In [ ]:
##print(len(x_deltaclose)+1, len(x_deltaclose)+1) #linee del csv utilizzate (pari al numero di punti plottati +1)
##numero dati totali e differenza in lunghezza tra le liste dati dei csv dei due titoli. Supponendo che 
##vi sia solo un offset temporale tra i dati dei due csv, la loro differenza deve coincidere con quanto sopra
##print(max(len(x_linelist), len(y_linelist)), abs(len(x_linelist) - len(y_linelist))) 